In [2]:
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

# for jupyter notebook
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
df = pd.read_csv('ICD9CM.csv.gz')

df[['Class ID', 'Preferred Label', 'Parents']].head()

,Class ID,Preferred Label,Parents
0,http://purl.bioontology.org/ontology/ICD9CM/83.72,Recession of tendon,http://purl.bioontology.org/ontology/ICD9CM/83.7
1,http://purl.bioontology.org/ontology/ICD9CM/987.8,"Toxic effect of other specified gases, fumes, ...",http://purl.bioontology.org/ontology/ICD9CM/987
2,http://purl.bioontology.org/ontology/ICD9CM/80.23,"Arthroscopy, wrist",http://purl.bioontology.org/ontology/ICD9CM/80.2
3,http://purl.bioontology.org/ontology/ICD9CM/346.4,Menstrual migraine,http://purl.bioontology.org/ontology/ICD9CM/346
4,http://purl.bioontology.org/ontology/ICD9CM/65...,"Chromosomal abnormality in fetus, affecting ma...",http://purl.bioontology.org/ontology/ICD9CM/655.1


## Search data

In [ ]:
df['Class ID'] = df['Class ID'].str.lower()

df[df['Class ID'].str.contains("/0.?1.?1.?6.?6", na=False)]['Preferred Label'].values

# Split the URL to get codes

In [4]:
def splitICD9(line):
    if isinstance(line, str):
        return line.rsplit('/', 1)[1]
    return line

In [5]:
# Only keep the notation 
df['Class ID'] = df['Class ID'].progress_map(lambda x: splitICD9(x))
df['Parents'] = df['Parents'].progress_map(lambda x: splitICD9(x))

100%|████████████████████████████████| 22533/22533 [00:00<00:00, 563268.46it/s]


# Make dataframe containing codes with all the parents

First we need to figure out all the super parents and stop adding parents til we find them. Current super parents found are following. The first four are marked with the label owl#Thing

* '001-99999': 'DISEASES AND INJURIES',
* '00-9999': 'PROCEDURES',
* 'E000-E9999':'SUPPLEMENTARY CLASSIFICATION OF EXTERNAL CAUSES OF INJURY AND POISONING',
* 'V01-V9199':'SUPPLEMENTARY CLASSIFICATION OF FACTORS INFLUENCING HEALTH STATUS AND CONTACT WITH HEALTH SERVICES',
* 'T051': 'Event',
* 'T071': 'Entity'

In [6]:
last = ['owl#Thing', 'T051', 'T071']

In [7]:
ICD9_df = pd.DataFrame()

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    
    # code to the row
    current = row
    tree = []
    tree.append(current['Class ID'])
 
    # add patents to the row
    while (current['Parents'] not in last) and (current['Class ID'] not in last):
        current = df.loc[df.index[df['Class ID'] == current['Parents']][0]]
        tree.append(current['Class ID'])
    
    # Add row to the dataframe
    if ICD9_df.empty:
        ICD9_df = pd.DataFrame([tree])
    else:
        ICD9_df = ICD9_df.append([tree], ignore_index=True)


  0%|                                                | 0/22533 [00:00<?, ?it/s]C:\Users\sohaibn\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)
100%|████████████████████████████████████| 22533/22533 [03:46<00:00, 85.58it/s]


In [8]:
# remove duplicated parents (comment out for labeling data, not for semantics)
#ICD9_df = ICD9_df[~ICD9_df[0].isin(ICD9_df[1])]

# fill in nan with last parent
ICD9_df = ICD9_df.fillna(method='ffill', axis=1)

In [9]:
ICD9_df.to_csv('ICD9_CODES.csv',index=False)

# Split between root parents

In [10]:
ICD9_df = pd.read_csv('ICD9_CODES.csv')

In [11]:
# SPLIT BETWEEN DIADNOSE AND PROCEDURES

df_dis = pd.DataFrame()
df_proc = pd.DataFrame()
df_supp_ext = pd.DataFrame()
df_supp_fact = pd.DataFrame()

for index, row in tqdm(ICD9_df.iterrows(), total=ICD9_df.shape[0]):
    
    # get superclass
    index = row.last_valid_index()
    label = row[index]
    
    # create diagnose dataframe
    if label == '001-999.99':
        if df_dis.empty:
            df_dis = pd.DataFrame([row])
        else:
            df_dis = df_dis.append([row],ignore_index=True)
    
    # create procedures dataframe
    if label == '00-99.99':
        if df_proc.empty:
            df_proc = pd.DataFrame([row])
        else:
            df_proc = df_proc.append([row],ignore_index=True)
            
    # create procedures dataframe
    if label == 'E000-E999.9':
        if df_supp_ext.empty:
            df_supp_ext = pd.DataFrame([row])
        else:
            df_supp_ext = df_supp_ext.append([row],ignore_index=True)
            
    # create procedures dataframe
    if label == 'V01-V91.99':
        if df_supp_fact.empty:
            df_supp_fact = pd.DataFrame([row])
        else:
            df_supp_fact = df_supp_fact.append([row],ignore_index=True)

100%|███████████████████████████████████| 22533/22533 [00:49<00:00, 459.05it/s]


In [12]:
df_dis.to_csv('ICD9_DIS_CODE.csv', index=False)
df_proc.to_csv('ICD9_PROC_CODE.csv', index=False)
df_supp_ext.to_csv('ICD9_SUPP_EXT_CODE.csv', index=False)
df_supp_fact.to_csv('ICD9_SUPP_FACT_CODE.csv', index=False)

In [ ]:
# 407 not included for some reason
